In [0]:
a = 1
while a == 1:
  a = 1

KeyboardInterrupt: ignored

In [0]:
import zipfile
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer 
import gensim 
from gensim.models import Word2Vec
from tqdm import tqdm
from textblob import TextBlob
import json

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/DATASET"

full_dataset.csv  language.csv	v-1.1.1-single-h-NER-incomplete.zip


In [0]:
!cp "/content/drive/My Drive/DATASET/full_dataset.csv" "recipes.csv"

In [0]:
recipes = pd.read_csv('full_dataset.csv', encoding='utf8') 

In [0]:
ner = pd.read_csv('NER.csv', encoding='utf8') 
ner

,Unnamed: 0,link,NER
0,0,www.cookbooks.com/Recipe-Details.aspx?id=44874,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,www.cookbooks.com/Recipe-Details.aspx?id=699419,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,www.cookbooks.com/Recipe-Details.aspx?id=10570,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,www.cookbooks.com/Recipe-Details.aspx?id=897570,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,www.cookbooks.com/Recipe-Details.aspx?id=659239,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...
2231137,2231137,www.foodnetwork.com/recipes/sunny-anderson/sun...,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,2231138,cookpad.com/us/recipes/355411-devil-eggs,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,2231139,cookpad.com/us/recipes/153324-extremely-easy-a...,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,2231140,cooking.nytimes.com/recipes/1015164,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [0]:
recipes = recipes.drop(recipes.columns[[0]], axis=1)
recipes

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [0]:
nan_rows = recipes[recipes['NER'].isnull()]

In [0]:
nan_rows

,title,ingredients,directions,link,source,NER
1011,Chicken Broccoli Casserole,"[""1 pkg. Stove Top stuffing, cooked""]","[""Sprinkle top with 1 cup grated cheese."", ""Ba...",www.cookbooks.com/Recipe-Details.aspx?id=787695,Gathered,NaN
1138,Chicken Casserole,"[""4 to 6 large chicken breasts"", ""1 can cream ...","[""Add can of chicken broth to chicken while co...",www.cookbooks.com/Recipe-Details.aspx?id=115399,Gathered,NaN
1290,Baklava,"[""10 oz. walnuts, finely chopped"", ""7 oz. almo...","[""Mix together nuts, cinnamon, cloves and suga...",www.cookbooks.com/Recipe-Details.aspx?id=977518,Gathered,NaN
1448,Pork Shoulder Roast,"[""roast (3 to 5 lb.)""]","[""Sprinkle roast with some tenderizer, salt an...",www.cookbooks.com/Recipe-Details.aspx?id=67832,Gathered,NaN
1833,Chicken Pie,"[""3 to 4 lb. hen or 6 chicken breasts"", ""1 to ...","[""Cook chicken until tender; remove all bones ...",www.cookbooks.com/Recipe-Details.aspx?id=1030333,Gathered,NaN
...,...,...,...,...,...,...
2231080,Konnyaku Okonomiyaki,"[""1/2 Konnyaku"", ""1/4 Cabbage"", ""3 tbsp Flour ...","[""Cut the konnyaku into 3 mm thick slices, the...",cookpad.com/us/recipes/158958-konnyaku-okonomi...,Recipes1M,NaN
2231087,Black Bean Chilaquiles Recipe,"[""12 x Corn Tortillas"", ""1 x Onion minced"", ""1...","[""1."", ""Pre-heat oven to 300 degrees F, 150 de...",cookeatshare.com/recipes/black-bean-chilaquile...,Recipes1M,NaN
2231108,Can't Wait Tailgate BBQ Tacos Recipe alender,"[""1 pkg. angelhair or finely shredded cole sla...","[""combine the first 8 ingredients in a large b...",www.chowhound.com/recipes/wait-tailgate-bbq-ta...,Recipes1M,NaN
2231126,Cucumber Sandwiches,"[""1 cucumber, lightly peeled"", ""1/2 tsp salt"",...","[""Prepare ahead"", ""Step 1 can be completed 1 h...",www.cookstr.com/recipes/cucumber-sandwiches,Recipes1M,NaN


In [0]:
nan_idx = nan_rows.index
nan_idx

Int64Index([   1011,    1138,    1290,    1448,    1833,    1835,    2330,
               2531,    2547,    3771,
            ...
            2230834, 2230895, 2230948, 2230967, 2231011, 2231080, 2231087,
            2231108, 2231126, 2231138],
           dtype='int64', length=34970)

In [0]:
recipes.describe()

,title,ingredients,directions,link,source,NER
count,2231142,2231142,2231142,2231142,2231142,2231142
unique,1312871,2226362,2211644,2231142,2,2133496
top,Chicken Casserole,"[""1 c. peanut butter"", ""1 c. sugar"", ""1 egg""]","[""Mix all ingredients together.""]",www.foodnetwork.com/recipes/crawfish-pie-recip...,Gathered,[]
freq,4099,28,274,1,1643098,573


In [0]:
r_ingredients = []
for i in tqdm(range(0, len(recipes))):
  r_ingredients.append(json.loads(recipes.ingredients[i])) 

100%|██████████| 2231142/2231142 [01:26<00:00, 25696.26it/s]


In [0]:
model = 'model'
if model is not None:
	nlp = spacy.load(model)

In [0]:
def num_there(s):
    return any(i.isdigit() for i in s)

In [0]:
for i in tqdm(range(len(nan_idx))):
  idx = nan_idx[i]
  ing = []
  for j in range(0, len(r_ingredients[idx])):
      doc = nlp(r_ingredients[idx][j])
      if len(doc.ents)>0 and num_there(str(doc.ents[0])) == False:
        ing.append(str(doc.ents[0]))
  x = json.dumps(ing)
  recipes.NER[idx] = x

100%|██████████| 34970/34970 [32:25<00:00, 17.97it/s]


In [0]:
recipes

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [0]:
df_new = pd.DataFrame(columns=['link', 'NER'], data=recipes[['link','NER']].values)

In [0]:
df_new

,link,NER
0,www.cookbooks.com/Recipe-Details.aspx?id=44874,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,www.cookbooks.com/Recipe-Details.aspx?id=699419,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,www.cookbooks.com/Recipe-Details.aspx?id=10570,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,www.cookbooks.com/Recipe-Details.aspx?id=897570,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,www.cookbooks.com/Recipe-Details.aspx?id=659239,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...
2231137,www.foodnetwork.com/recipes/sunny-anderson/sun...,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,cookpad.com/us/recipes/355411-devil-eggs,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,cookpad.com/us/recipes/153324-extremely-easy-a...,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,cooking.nytimes.com/recipes/1015164,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [0]:
recipes.to_csv('full_dataset.csv')

In [0]:
df_new.to_csv('NER.csv')

In [0]:
x = json.loads(NER[1])
x[1]

'chicken breasts'

In [0]:
remove = recipes.loc[recipes.NER.map(lambda x: len(json.loads(x)) > 1]

SyntaxError: ignored

In [0]:
for i in tqdm(range(0, len(recipes))):

  ing = []
  for j in range(0, len(r_ingredients[i])):
    doc = nlp(r_ingredients[i][j])
    if len(doc.ents) > 0 and num_there(str(doc.ents[0])) == False:
      ing.append(str(doc.ents[0]))
  x = json.dumps(ing)
  recipes.NER[i] = x

In [0]:
remove

In [0]:
recipes = pd.concat([recipes, remove]).drop_duplicates(keep=False)
recipes.reset_index(drop=True, inplace=True)

In [0]:
!cp "/content/drive/My Drive/DATASET/DATASET.csv" "recipes.csv"